In [0]:
print("Hello world!")

In [0]:
tpch_path = '/dbfs/databricks-datasets/tpch'

In [0]:
with open(tpch_path + "/README.md" , 'r') as f:
     content = f.read()
displayHTML(f"<pre>{content}</pre>")

In [0]:
table_names = [table.name for table in spark.catalog.listTables("samples.tpch")]

In [0]:
for table_name in table_names:
    print("Table name: " + table_name)
    spark.sql(f"DESCRIBE samples.tpch.{table_name}").show()

In [0]:
from dlt import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Pipeline configuration - creates catalog and schemas automatically
@dlt.create_table(
    comment="Raw customer data in bronze layer",
    table_properties={
        "quality": "bronze",
        "pipelines.autoOptimize.managed": "true"
    }
)
def bronze_customer():
    """
    Bronze layer: Raw data copy from samples.tpch.customer
    """
    return (
        spark.table("samples.tpch.customer")
        .select(
            col("c_custkey"),
            col("c_name"),
            col("c_address"),
            col("c_nationkey"),
            col("c_phone"),
            col("c_acctbal"),
            col("c_mktsegment"),
            col("c_comment"),
            current_timestamp().alias("_load_timestamp"),
            lit("samples.tpch.customer").alias("_source")
        )
    )